In [2]:
from model.hierarchical_self_attention_model import HSA_model
import tensorflow as tf
import yaml

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
hparam_file = open('configs/hyperparameters.yaml', mode='r')
hyperparameters = yaml.load(hparam_file, Loader=yaml.FullLoader)

In [10]:
hyperparameters

{'HSA_model': {'modality_indices': {'opp': [0, 9, 18, 27, 36, 45, 61, 77],
   'mex': [0, 3, 6]},
  'dff': 512,
  'd_model': 64,
  'num_heads': 4,
  'dropout': 0.2}}

In [11]:
n_window, n_timesteps, n_features, n_outputs = 36, 25, 77, 6

hsa_model = HSA_model(hyperparameters['HSA_model']['modality_indices']['opp'], n_window, n_timesteps, n_features, n_outputs, d_model=hyperparameters['HSA_model']['d_model'], num_heads = hyperparameters['HSA_model']['num_heads'], dff=hyperparameters['HSA_model']['dff'], dropout_rate=hyperparameters['HSA_model']['dropout'])

model = hsa_model.get_model()

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics='accuracy')
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36, 25, 77)]      0         
_________________________________________________________________
multi_window_encoder (MultiW ((None, None, 64), (None, 1249472   
_________________________________________________________________
modality_encoder_block_7 (Mo (None, None, 64)          170048    
_________________________________________________________________
combined_sensor_self_attenti ((None, 64), (None, 1, No 82880     
_________________________________________________________________
dense_108 (Dense)            (None, 6)                 390       
Total params: 1,502,790
Trainable params: 1,502,790
Non-trainable params: 0
_________________________________________________________________
